In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, LeaveOneOut, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score,classification_report, f1_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, OneHotEncoder, LabelEncoder, PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings("ignore")
from statistics import mean
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.metrics import make_scorer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
import plotly.express as px
from sklearn.model_selection import learning_curve
%run auxiliar.py

# Definir la metrica a utilizar 


In [11]:
scorer = metrics.make_scorer(metrics.f1_score)## Modelado

In [12]:
# Lectura de los datos
data = pd.read_csv("../data/superstore_clean.csv")

In [13]:
df_limpio = data.copy()

In [14]:
list(data.columns.values)

['Education_Graduation',
 'Education_PhD',
 'Education_Master',
 'Education_Basic',
 'Marital_Status_Divorced',
 'Marital_Status_Single',
 'Marital_Status_Married',
 'Income',
 'Kidhome',
 'Teenhome',
 'Recency',
 'MntWines',
 'MntFruits',
 'MntMeatProducts',
 'MntFishProducts',
 'MntSweetProducts',
 'MntGoldProds',
 'NumDealsPurchases',
 'NumWebPurchases',
 'NumCatalogPurchases',
 'NumStorePurchases',
 'NumWebVisitsMonth',
 'Response',
 'Complain',
 'Edad',
 'Dt_Customer_year',
 'tiempo_participacion']

In [53]:
# # Solo con columnas seleccionadas en proceso de seleccion de variables
predictors = df_limpio[[
    "Recency","NumCatalogPurchases", "NumStorePurchases", "MntMeatProducts", "MntWines","NumWebVisitsMonth", "MntGoldProds", "Income", "tiempo_participacion"
    
]]

In [54]:
# X = predictors
X = predictors
y = df_limpio.Response

In [55]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 12345, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1565, 9), (671, 9), (1565,), (671,))

In [56]:
%%time
# Definir modelo
cbc = CatBoostClassifier(logging_level='Silent', random_state=12345)

# Crear la grilla
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300],'learning_rate' : [0.01, 0.05, 0.1]}

#cv
cv = StratifiedKFold(n_splits=5, shuffle= True, random_state=12345)

# Llama a grid search
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring = scorer, cv = cv, verbose=False)

#fit the model
gscv.fit(X_train,y_train)

# Resultados
print(f'Mejor estimador: {gscv.best_estimator_}')
print(f'Mejores parametros: {gscv.best_params_}')
print(f'Mejor score: {gscv.best_score_}')


Mejor estimador: <catboost.core.CatBoostClassifier object at 0x7f9872a5c5b0>
Mejores parametros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300}
Mejor score: 0.45301186240244845
CPU times: user 30.2 s, sys: 23.8 s, total: 54 s
Wall time: 20.8 s


In [64]:
catboost_9 = CatBoostClassifier(
    max_depth = 4,
    n_estimators = 300,
    learning_rate = 0.1,
    logging_level = 'Silent' #para que no salga print de iteraciones

)

catboost_9.fit(X_train, y_train,eval_set=(X_test, y_test),early_stopping_rounds = 30,use_best_model = True)

In [65]:
# Resultados en train
res_cat = reportes(catboost_9, X_train, y_train, nombre = "cat train")
res_cat

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train,0.9393,0.6111,0.8041,0.9728,0.7507


In [66]:
res_cat_test = reportes(catboost_9, X_test, y_test, nombre = "cat test")
res_cat_test

,Accuracy,Recall,Roc-Auc,Precision,F1
cat test,0.8584,0.26,0.6116,0.5532,0.3537


In [67]:
catboost_t = CatBoostClassifier(
    max_depth = 4,
    n_estimators = 300,
    learning_rate = 0.1,
    logging_level = 'Silent', #para que no salga print de iteraciones
    scale_pos_weight=3,

)

catboost_t.fit(X_train, y_train,eval_set=(X_test, y_test),early_stopping_rounds = 30,use_best_model = True)

In [68]:
# Resultados en train
res_cat_3 = reportes(catboost_t, X_train, y_train, nombre = "cat train scale")
res_cat_3

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train,0.8888,0.7222,0.8202,0.6079,0.6602


In [71]:
# Resultados en train
res_cat_3_t = reportes(catboost_t, X_test, y_test, nombre = "cat train scale")
res_cat_3_t

,Accuracy,Recall,Roc-Auc,Precision,F1
cat train scale,0.8644,0.55,0.7347,0.5446,0.5473


In [72]:
import pickle
# Guarda el modelo
pickle.dump(catboost_t, open('../web/model_9.pkl','wb'))